* Necessary Libraries

In [15]:
import os
import pandas as pd
import numpy as np
from math import sqrt

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


Read in data 

In [16]:
# read text file into pyspark dataframe
#df = spark.read.csv("ds5559/FinalProject/ComboCities.csv",  inferSchema=True, header = True)
df = pd.read_csv("ds5559/FinalProject/ComboCities.csv")

In [17]:
df.head()

,City,All industry total (in thousands),Private industries,"Agriculture, forestry, fishing and hunting","Mining, quarrying, and oil and gas extraction",Utilities,Construction,Manufacturing,Durable goods manufacturing,Nondurable goods manufacturing,...,NFL Team relocated,NBA Team Numbers,NBA Team Value,NBA Team Revenue,Large Market,Medium Market,Small Market,No Teams,Year,index
0,Atlanta,209741698.0,191749470.0,366262.00,331505.00,2100718.00,(D),(D),(D),(D),...,0.0,1.0,199000.0,76000.0,1.0,0.0,0.0,0.0,2001.0,1.0
1,Austin,55307638.0,47708348.0,21993.00,205947.00,381802.00,4324915.00,7829516.00,(D),(D),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2001.0,2.0
2,Boise,16435618.0,14448912.0,(D),13511.00,57379.00,1174578.00,3196453.00,(D),(D),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2001.0,3.0
3,Boston,236051306.0,213403308.0,(D),87120.00,2666446.00,9560212.00,26949729.00,(D),(D),...,0.0,1.0,218000.0,81000.0,0.0,1.0,0.0,0.0,2001.0,4.0
4,Charlotte,76027214.0,69766921.0,(D),105937.00,1805777.00,3895799.00,14555666.00,(D),(D),...,0.0,1.0,135000.0,65000.0,0.0,0.0,1.0,0.0,2001.0,5.0


Initial review of data

In [18]:
len(df)

432

In [19]:
len(df.columns)

95

In [20]:
df.dtypes

City                                                  object
All industry total (in thousands)                    float64
  Private industries                                 float64
    Agriculture, forestry, fishing and hunting        object
    Mining, quarrying, and oil and gas extraction     object
                                                      ...   
Medium Market                                        float64
Small Market                                         float64
No Teams                                             float64
Year                                                 float64
index                                                float64
Length: 95, dtype: object

In [21]:
df.columns

Index(['City', 'All industry total (in thousands)', '  Private industries',
       '    Agriculture, forestry, fishing and hunting',
       '    Mining, quarrying, and oil and gas extraction', '    Utilities',
       '    Construction', '    Manufacturing',
       '      Durable goods manufacturing',
       '      Nondurable goods manufacturing', '    Wholesale trade',
       '    Retail trade', '    Transportation and warehousing',
       '    Information',
       '    Finance, insurance, real estate, rental, and leasing',
       '      Finance and insurance',
       '      Real estate and rental and leasing',
       '    Professional and business services',
       '      Professional, scientific, and technical services',
       '      Management of companies and enterprises',
       '      Administrative and support and waste management and remediation services',
       '    Educational services, health care, and social assistance',
       '      Educational services', '      Health 

Questions to answer

# 1) Does a city having a sports team affect its GDP or unemployment. Regression 

# 2) Does market size matter to economic impact?

# 3) The year the team joins a league or when ownership changes?


